# Bioproject y bases de datos relacionadas

Hay un conjunto de bases de datos que almacenan información sobre proyectos de secuenciación masiva de ácidos nucleicos. Que incluyen entre otros:

- Genómica
- Transcriptómica
- GWAS
- Análisis de variantes
- Metagenómica

Cada una de estas bases de datos se concentran en un aspecto diferente de los datos.


- **BioProject**:
  - Es una base de datos que organiza y agrupa proyectos de investigación relacionados con datos biológicos.
  - Actúa como un contenedor que conecta múltiples BioSamples y datos asociados.
  - Proporciona un contexto general del estudio, incluyendo objetivos y metodología.

- **BioSample**:
  - Contiene información detallada sobre las muestras biológicas utilizadas en un proyecto.
  - Cada muestra está asociada a un BioProject y puede incluir datos como el organismo, condiciones experimentales y réplicas.
  - Sirve como puente entre los datos experimentales y los metadatos biológicos.

- **SRA (Sequence Read Archive)**:
  - Almacena datos de secuenciación masiva generados en proyectos de investigación.
  - Cada conjunto de datos en SRA está vinculado a un BioSample y, por extensión, a un BioProject.
  - Permite el acceso a datos crudos de secuencias para análisis posteriores.


![SRA](https://github.com/javieriserte/bases-de-datos-unq-unnoba/raw/main/images/sra.svg)

### Recuperar los Biosample ids de un Bioproject

Para acceder a los identificadores de los Biosample asocicados con un bioproject
necesitamos usar el acceso ELink de Entrez/eUtils.

In [10]:
import json
import requests
# Define el ID del BioProject
bioproject_id = "PRJNA1012728"  # Reemplaza con el ID de tu BioProject
uid = "1012728" # El mismo bioproject_id sin el prefijo PRJNA

# URL de la API de E-utilities para buscar en BioSample
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi"

params = {
    "dbfrom": "bioproject",
    "db": "biosample",
    "id": uid,
    "retmode": "json"
}

# Realiza la solicitud a la API
response = requests.get(url, params=params, timeout=10)

response.raise_for_status()

data = response.json()


In [12]:

print(json.dumps(data, indent=2))

{
  "header": {
    "type": "elink",
    "version": "0.3"
  },
  "linksets": [
    {
      "dbfrom": "bioproject",
      "ids": [
        "1012728"
      ],
      "linksetdbs": [
        {
          "dbto": "biosample",
          "linkname": "bioproject_biosample_sp",
          "links": [
            "37271905",
            "37271904",
            "37271903",
            "37271902",
            "37271901",
            "37271900",
            "37271899",
            "37271898",
            "37271897",
            "37271896",
            "37271895",
            "37271894"
          ]
        },
        {
          "dbto": "biosample",
          "linkname": "bioproject_biosample",
          "links": [
            "37271905",
            "37271904",
            "37271903",
            "37271902",
            "37271901",
            "37271900",
            "37271899",
            "37271898",
            "37271897",
            "37271896",
            "37271895",
            "37271894"
     

In [18]:
import requests
import jsonpath_ng.ext as jp

linksets = [x.value for x in jp.parse("$..linksetdbs[*]").find(data)]
biosample_ids = [
    biosample_id
    for linkset in linksets
    if linkset.get("linkname", "") == "bioproject_biosample"
    for biosample_id in linkset.get("links", [])
]
biosample_ids

['37271905',
 '37271904',
 '37271903',
 '37271902',
 '37271901',
 '37271900',
 '37271899',
 '37271898',
 '37271897',
 '37271896',
 '37271895',
 '37271894']

### Recuperar los SRA Ids de un Bioproject

In [3]:
import requests
import jsonpath_ng.ext as jp

# Define el ID del BioProject
bioproject_id = "PRJNA12345"  # Reemplaza con el ID de tu BioProject

# URL de la API de E-utilities para buscar en SRA
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"

params = {
    "db":"sra",
    "term": "PRJNA12345[BioProject]",
    "retmode":"json"
}

# Realiza la solicitud a la API
response = requests.get(url, params=params, timeout=10)

response.raise_for_status()

data = response.json()
ids = [x.value for x in jp.parse("$..idlist[*]").find(data)]
print(ids)

['11694043', '11694042', '11694041', '11694040', '11694039', '7520472', '7520471', '7520470', '7520469', '7520468', '7520467', '7520466', '7520465', '1561605', '1105296']


### Recuperar el accession de un SRA RUN por su Id

Si bien el SRA Id y el SRA Accession hacen referencia a una misma entidad
en muchas aplicaciones es necesario trabajar con los Accession, por lo que
tendremos que mapear los ids al accession que le corresponde. Esto lo podemos
hacer usando el accesso ESummary de Entrez/eUtils.

Tambien lo podemos recuperar usando eFetch, pero procesar el resultado es
mucho más complicado.

In [21]:

bioproject_id = "PRJNA12345"  # Reemplaza con el ID de tu BioProject

# URL de la API de E-utilities para buscar en SRA
url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"

params = {
    "id": ",".join(ids),
    "db":"sra",
    "retmode":"json"
}

# Realiza la solicitud a la API
response = requests.get(url, params=params, timeout=10)

response.raise_for_status()

data = response.json()

print(json.dumps(data, indent=2))

{
  "header": {
    "type": "esummary",
    "version": "0.3"
  },
  "result": {
    "uids": [
      "11694043",
      "11694042",
      "11694041",
      "11694040",
      "11694039",
      "7520472",
      "7520471",
      "7520470",
      "7520469",
      "7520468",
      "7520467",
      "7520466",
      "7520465",
      "1561605",
      "1105296"
    ],
    "11694043": {
      "uid": "11694043",
      "expxml": "  <Summary><Title>microbical community</Title><Platform instrument_model=\"Illumina HiSeq 1000\">ILLUMINA</Platform><Statistics total_runs=\"1\" total_spots=\"37238\" total_bases=\"22417276\" total_size=\"13019893\" load_done=\"true\" cluster_name=\"public\"/></Summary><Submitter acc=\"SRA1116229\" center_name=\"yangzhou university\" contact_name=\"jie chen zhu\" lab_name=\"environment science and enginerring\"/><Experiment acc=\"SRX8997589\" ver=\"1\" status=\"public\" name=\"microbical community\"/><Study acc=\"SRP057140\" name=\"Effects of hypomagnetic fields and simulat

In [22]:
import re

# Necesito buscar los SRR con una expresion regular.
pattern = re.compile('acc="(SRR[0-9]+)"')
for i in ids:
    c_runs = jp.parse(f'$..["{i}"].runs').find(data)[0].value
    matches = re.findall(pattern, c_runs)
    for m in matches:
        print(f"UID: {i}, Accession: {m}")

UID: 11694043, Accession: SRR12506406
UID: 11694042, Accession: SRR12506407
UID: 11694041, Accession: SRR12506408
UID: 11694040, Accession: SRR12506409
UID: 11694039, Accession: SRR12506410
UID: 7520472, Accession: SRR8781789
UID: 7520471, Accession: SRR8781790
UID: 7520470, Accession: SRR8781791
UID: 7520469, Accession: SRR8781792
UID: 7520468, Accession: SRR8781793
UID: 7520467, Accession: SRR8781794
UID: 7520466, Accession: SRR8781795
UID: 7520465, Accession: SRR8781796
UID: 1561605, Accession: SRR2080773
UID: 1105296, Accession: SRR1659097


## GEO Datasets (GSE)

- Contienen experimentos completos de expresión génica, como microarrays o RNA-seq.
- Cada dataset se identifica con un ID tipo GSEXXXX.
- Incluye varias muestras (cada una con un GSM), información de diseño experimental, y a menudo está enlazado con un BioProject.

## GEO Profiles

- Son perfiles individuales de expresión de un gen extraídos de los GEO Datasets.
- Cada perfil (GDSXXXX) representa cómo se expresa un gen a lo largo de las muestras de un dataset.
- Es decir, 1 GSE (dataset) ➝ muchos GEO Profiles (uno por gen)

## Métodos de descargas

Los datos de secuenciación masiva ocupan una gran cantidad de espacio de almacenamiento.
Por ello, hay metodologías especiales para descargarlos:

- RUN selector
- SRA Toolkit

### RUN selector

El **RUN selector** del NCBI es una herramienta que permite explorar y
seleccionar datos de secuenciación asociados a un proyecto o estudio específico.
Proporciona una interfaz interactiva donde se pueden filtrar y visualizar los
datos disponibles, como las plataformas de secuenciación utilizadas, el número
de lecturas generadas, y el tamaño total de los datos.

Funciona de la siguiente manera:

1. **Búsqueda**: Se ingresa un identificador de proyecto (por ejemplo, un
   BioProject o BioSample) o palabras clave relacionadas.
2. **Visualización**: Muestra una tabla con los datos de secuenciación
   disponibles, incluyendo información como el ID del RUN (SRRXXXXXXX), la
   plataforma de secuenciación, y el tamaño de los datos.
3. **Selección**: Permite seleccionar los RUNs de interés para su descarga o
   análisis. La descarga directa por la web está limitada por el tamaño del
   archivo. Alternativamente se pueden enviar los datos a los servidores Cloud
   AWS, Google cloud o el servicio Galaxy.
4. **Exportación**: Se pueden exportar los IDs seleccionados para utilizarlos
   con herramientas como el SRA Toolkit.

Se puede acceder desde la página web de SRA en NCBI.

[RUN Selector](https://www.ncbi.nlm.nih.gov/Traces/study/)



### SRA Toolkit

El **SRA Toolkit** es un conjunto de herramientas:
- Descargar, procesar y convertir datos almacenados en el Sequence Read Archive (SRA).
- Estas herramientas permiten trabajar con grandes volúmenes de datos de secuenciación de manera eficiente.

#### Funcionalidades principales:
- **Descarga de datos**: Permite descargar archivos SRA directamente desde el repositorio.
- **Conversión de formatos**: Convierte archivos SRA a formatos más comunes como FASTQ para su análisis.
- **Gestión de datos**: Facilita la extracción y manipulación de datos específicos dentro de un archivo SRA.

#### Herramientas incluidas:
- `prefetch`: Descarga archivos SRA desde el repositorio.
- `fasterq-dump`: Convierte archivos SRA en archivos FASTQ de manera rápida y eficiente.
- `sam-dump`: Extrae datos en formato SAM para análisis de alineación.

Más información y descarga: [SRA Toolkit](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit)


### Comandos más relevantes:

  ```bash
  # Descargar el archivo SRA.
  > prefetch SRR0000001 -O /path/to/folder/
  # Convierte el archivo SRA en archivos fastq.
  > fasterq-dump /path/to/folder/SRR0000001.sra
  ```

### Formato FASTQ

El formato **FASTQ** es ampliamente utilizado para almacenar datos de secuenciación de ADN/ARN. Combina información de secuencia y calidad en un solo archivo. Cada entrada consta de cuatro líneas:

1. **Encabezado**: Comienza con `@` seguido de un identificador único para la secuencia.
2. **Secuencia**: La secuencia de nucleótidos (A, T, C, G, N).
3. **Separador**: Una línea que comienza con `+`, opcionalmente seguida del mismo identificador que en el encabezado.
4. **Calidad**: Una cadena de caracteres que representa la calidad de cada base en la secuencia (codificada en ASCII).

**Ejemplo**:
```
@SEQ_ID
GATTTGGGGTTTCCCAGTCACGAC
+
!''*((((***+))%%%++)(%%%
```

---

### Formato SAM

El formato **SAM** (Sequence Alignment/Map) se utiliza para almacenar alineaciones de secuencias contra un genoma de referencia. Es un formato tabular con dos secciones principales:

1. **Encabezado** (opcional): Comienza con `@` e incluye información sobre la referencia y el alineamiento.
2. **Cuerpo**: Contiene una fila por cada lectura alineada, con al menos 11 campos obligatorios y campos opcionales.

**Campos obligatorios**:
1. QNAME: Nombre de la secuencia de consulta.
2. FLAG: Indicadores sobre el estado de la alineación.
3. RNAME: Nombre de la referencia contra la que se alineó.
4. POS: Posición inicial de la alineación.
5. MAPQ: Calidad del alineamiento.
6. CIGAR: Descripción de la alineación (inserciones, deleciones, etc.).
7. RNEXT: Nombre de la referencia del siguiente segmento.
8. PNEXT: Posición del siguiente segmento.
9. TLEN: Longitud del fragmento.
10. SEQ: Secuencia de nucleótidos.
11. QUAL: Calidad de la secuencia.

**Ejemplo**:
```
@HD     VN:1.6  SO:coordinate
@SQ     SN:chr1 LN:248956422
read001  0       chr1    100     60      50M     *       0       0       AGCTTAGCTAGCTACCTATATCTTGGTCTTGGCCGATGCTTAGCTAG   * 
```

### Formato CIGAR

El **formato CIGAR** (Compact Idiosyncratic Gapped Alignment Report) es una
representación compacta que describe cómo una secuencia de consulta (query) se
alinea con una secuencia de referencia. Se utiliza principalmente en el formato
SAM/BAM para especificar las operaciones de alineación.

Cada cadena CIGAR está compuesta por una serie de operaciones, donde cada
operación incluye un número que indica la longitud y una letra que representa el
tipo de operación.

#### Operaciones principales en CIGAR:
- **M**: Match o mismatch (alineación de bases entre la consulta y la
  referencia).
- **I**: Inserción (bases presentes en la consulta pero no en la referencia).
- **D**: Deleción (bases presentes en la referencia pero no en la consulta).
- **N**: Skip (saltos en la referencia, típicamente intrones en alineaciones de
  ARN).
- **S**: Soft clipping (bases no alineadas al inicio o final de la consulta,
  pero presentes en la secuencia original).
- **H**: Hard clipping (bases no alineadas al inicio o final de la consulta y
  eliminadas de la secuencia original).
- **P**: Padding (relleno en alineaciones con múltiples secuencias).
- **=**: Match exacto (bases idénticas entre la consulta y la referencia).
- **X**: Mismatch (bases diferentes entre la consulta y la referencia).

#### Ejemplo de una cadena CIGAR:
```
50M10I5D
```
- `50M`: 50 bases alineadas (match o mismatch).
- `10I`: 10 bases insertadas en la consulta.
- `5D`: 5 bases eliminadas de la referencia.

#### Notas importantes:
- El formato CIGAR no especifica si un `M` es un match o un mismatch, solo
  indica que hay una alineación.

Este formato es esencial para interpretar cómo se mapean las lecturas a un
genoma de referencia y para realizar análisis posteriores como detección de
variantes o ensamblaje de genomas.